# Astronomical Coordinates 4: Working with Velocity Data in astropy.coordinates

## Authors
Adrian Price-Whelan

## Learning Goals
* TODO

## Keywords
coordinates


## Summary

TODO: this is 3rd tutorial in a series...

## Imports

We start by importing some general packages we will need below:

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file

---

## More than just sky positions: Including velocity information in `SkyCoord`

As we have seen above, the `SkyCoord` object can store scalars or arrays of positional coordinate information, can transform between different coordinate frames, and can cross-match sets of coordinates. But wait, there's more! `astropy.coordinates` also supports representing and transforming *velocity* information along with positions ([docs](http://docs.astropy.org/en/latest/coordinates/velocities.html)).

### Passing velocity data in to `SkyCoord`

Velocity data is passed in to `SkyCoord` in a very similar way to the positional coordinates demonstrated above. However, velocity data must be passed in using keyword arguments, meaning that you must write out the velocity component names, e.g.:

In [ ]:
SkyCoord(ra=10*u.deg, 
         dec=20*u.deg,
         pm_ra_cosdec=1*u.mas/u.yr,
         pm_dec=2*u.mas/u.yr)

In the above, we create a `SkyCoord` instance with sky position and proper motion data. The proper motion component names for ICRS coordinates are, by default, `pm_ra_cosdec` and `pm_dec` ("pm" for "proper motion"). You can also specify radial velocity data via the `radial_velocity` argument, e.g.:

In [ ]:
velocity_coord = SkyCoord(ra=10*u.deg, 
                          dec=20*u.deg,
                          pm_ra_cosdec=1*u.mas/u.yr,
                          pm_dec=2*u.mas/u.yr,
                          radial_velocity=100*u.km/u.s)
velocity_coord

A `SkyCoord` object with velocity data can be transformed to other frames just like the objects we are used to from above:

In [ ]:
velocity_coord.transform_to('galactic')

An important caveat, however, is that some reference frames require knowing distance in order to transform all velocity components - something we did not specify in the `velocity_coord` definition above:

In [ ]:
# THIS SHOULD ERROR:
velocity_coord.transform_to(altaz)

### Evolving coordinate positions between epochs

For nearby or fast-moving stars, a star's position could change appreciably between two well-spaced observations of the source. For such cases, it might be necessary to evolve the position of the star using the proper motion or velocity of the star in order to accurately predict its position at a new time. Below, we will demonstrate this using data from the Digitized Sky Survey (DSS; digital scans of photographic plates observed in the 1950s) and data from the *Gaia* mission (at epoch J2015.5).

The star HD 219829 has a very large proper motion, close to 0.5 arcsec/year. Between the DSS and *Gaia*, we therefore expect that the position of the star has changed by about 0.5 arcmin; Let's see if this is the case! To start, we will query the *Gaia* catalog to retrieve data for this star (but skip the cell below if you do not have an internet connection):

In [ ]:
hd219829_coord = SkyCoord(ra=349.72896716*u.deg, 
                          dec=5.40511585*u.deg,
                          distance=34.47896*u.pc,
                          pm_ra_cosdec=483.41659*u.mas/u.yr,
                          pm_dec=-114.86339*u.mas/u.yr,
                          obstime=Time('J2015.5'))

Note above that we specify a new keyword argument, `obstime`, to `SkyCoord`: This is the observation time of the coordinates, and will be important later when we evolve the position of this star using its proper motion.

We now have the position and proper motion for HD 219829. Let's now query the DSS to retrieve a FITS image of the field around this star, using the STSCI DSS image cutout service. Again, skip the cell below if you do not have an internet connection:

In [ ]:
dss_cutout_filename = 'dss_hd219829.fits'

We can now load the FITS image of the cutout and use `astropy.visualization` to display the image using its World Coordinate System (WCS) info ([docs](http://docs.astropy.org/en/latest/visualization/wcsaxes/index.html)). By passing in the WCS information (included in the FITS cutout header), we can over-plot a marker for the *Gaia*-measured sky position of HD 219829:

In [ ]:
from astropy.wcs import WCS

hdu = fits.open(dss_cutout_filename)[0]
wcs = WCS(hdu.header)

fig, ax = plt.subplots(1, 1, figsize=(8, 8), 
                       subplot_kw=dict(projection=wcs))
ax.imshow(hdu.data, origin='lower', cmap='Greys_r')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
ax.set_autoscale_on(False)

ax.scatter(hd219829_coord.ra.degree,
           hd219829_coord.dec.degree,
           s=500,
           transform=ax.get_transform('world'),
           facecolor='none', linewidth=2, color='tab:red')

The bright star (as observed by DSS) is our target, and the red circle is where *Gaia* observed this star: It has moved! But, we can account for this, and predict the position of the star at around the time the DSS plate was observed (let's assume it was exactly 1950 - this is not strictly correct, but should be good enough).

To account for the proper motion of the source and evolve the position to a new time, we can use the `SkyCoord.apply_space_motion()` method ([docs](http://docs.astropy.org/en/latest/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord.apply_space_motion)):

In [ ]:
hd219829_coord_1950 = hd219829_coord.apply_space_motion(new_obstime=Time('J1950'))  # ignore the ErfaWarning

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8), 
                       subplot_kw=dict(projection=wcs))
ax.imshow(hdu.data, origin='lower', cmap='Greys_r')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
ax.set_autoscale_on(False)

ax.scatter(hd219829_coord.ra.degree,
           hd219829_coord.dec.degree,
           s=500,
           transform=ax.get_transform('world'),
           facecolor='none', linewidth=2, color='tab:red')

# Plot the predicted (past) position:
ax.scatter(hd219829_coord_1950.ra.degree,
           hd219829_coord_1950.dec.degree,
           s=500,
           transform=ax.get_transform('world'),
           facecolor='none', linewidth=2, color='tab:blue')

The predicted position of the star from 1950 is much closer to where the star is in the DSS image: Success!

### Exercises

The *Gaia* data table we downloaded for sources around NGC 188 also contains proper motion information. Do any of the sources in the field around NGC 188 have large enough proper motions that we might detect a difference between the DSS position of a star and the *Gaia* position? How many stars have an apparent change in position > 5 arcsec over 60 years?

In [ ]:
ngc188_pm = np.sqrt(table['pmra']**2 + table['pmdec']**2)
((ngc188_pm * 60*u.year) > 5*u.arcsec).sum()

Download a DSS image for the region around the fastest-moving star in the NGC 188 field. By accounting for the proper motion, can you predict where the star is in the DSS image using only the *Gaia* position and proper motion?

## Wrap-up

This tutorial series has covered a lot of material, but `astropy.coordinates` has even more functionality that we were unable to cover in this workshop. For documentation on other features of `astropy.coordinates`, check out [the astropy.coordinates section of the Astropy documentation](http://astropy.readthedocs.org/en/stable/coordinates/index.html).

You might also be interested in [the astroplan affiliated package](http://astroplan.readthedocs.org/), which uses the `astropy.coordinates` to do more advanced versions of the observation planning tasks briefly covered above.